# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Apache Cassandra coding portion of your project. 

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS data_modeling_project 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('data_modeling_project')
except Exception as e:
    print(e)

#### Read CSV file into a dataframe

In [ ]:
data = pd.read_csv('event_datafile_new.csv')
data.columns

#### Create pandas factory for converting Cassandra query results to df

In [ ]:
# Note I was trying to figure out a simple way to convert Cassandra query results to a dataframe and 
# found this on StackOverflow so I am attributing this
#https://stackoverflow.com/questions/41247345/python-read-cassandra-data-into-pandas
# This is the only thing I looked up during this project. It's pretty slick. 
    
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory

# Query 1
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4



#### Create Table

In [ ]:
# For this query I'll create a table with a composite key of sessionId and itemInSession
# Note I am not choosing to store all the data in this table but you could easily add the 
# other attributes

table_name = 'songs_by_session'

try:
    session.execute(f'drop table {table_name}')
except Exception as e:
    pass

query = f"create table if not exists {table_name} \
    (sessionId int, itemInSession int, artist varchar, song varchar, length float, \
    PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query) 
    print(f'Table {table_name} successfully created')
except Exception as e:
    print(e)

#### Insert Data

In [ ]:
query = f"INSERT INTO {table_name} \
        (sessionId, itemInSession, artist, song, length) \
        VALUES (%s, %s, %s, %s, %s)"

for row in data.itertuples():
        try:
            session.execute(query, (int(row.sessionId), 
                                    int(row.itemInSession), 
                                    row.artist, 
                                    row.song, 
                                    float(row.length)))
        except Exception as e:
            print(e)    

#### Select our results

In [ ]:
session_id = 338
item_id = 4

query = f'select artist, song, length from {table_name} where \
sessionId = {session_id} and itemInSession = {item_id}'

try:
    rows = session.execute(query)
except Exception as e:    
    print(e)

rows._current_rows

# Query 2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Create Table

In [ ]:
# For this query we'll create a table with a composite key of sessionId, userId, and itemInSession
# Note I am not choosing to store all the data in this table but you could easily add the 
# other attributes

table_name = 'songs_by_session_and_user'

try:
    session.execute(f'drop table {table_name}')
except Exception as e:
    pass

query = f"create table if not exists {table_name} \
    (sessionId int, userId int, itemInSession int,artist varchar, song varchar, user varchar, \
    PRIMARY KEY (sessionId, userId, itemInSession))"

try:
    session.execute(query) 
    print(f'Table {table_name} successfully created')
except Exception as e:
    print(e)
                        

#### Insert Data

In [ ]:
query = f"INSERT INTO {table_name} \
        (sessionId, userId, itemInSession, artist, song, user) \
        VALUES (%s, %s, %s, %s, %s, %s)"

for row in data.itertuples():
        try:
            session.execute(query, (int(row.sessionId),
                                    int(row.userId),
                                    int(row.itemInSession), 
                                    row.artist, 
                                    row.song, 
                                    row.firstName + " " + row.lastName))
        except Exception as e:
            print(e)    
            

#### Select our results

In [ ]:
user_id=10
session_id=182

query = f"select artist, song, user from {table_name} where \
    userId={user_id} and sessionId={session_id}"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
rows._current_rows

### Query 3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Create Table

In [ ]:
# For this query we'll create a table with a composite key of songtitle and userId . Note we
# need userID to make primary key unique since multiple users can listen to the same song.
# also note the same user could listen to the same song in multiple sessions, but since we don't 
# care about that we can choose to just have a primary key on these two fields.  Otherwise
# we could get duplicates in our query.  
# Also note I am choosing to store minimum data that we need for this query.  In reality I 
# would probabably store more attributes - better to have them then have to add them later.

table_name = 'songs_by_title'

try:
    session.execute(f'drop table {table_name}')
except Exception as e:
    pass

query = f"create table if not exists {table_name} \
    (song varchar, userId int, user varchar, \
    PRIMARY KEY (song, userId))"

try:
    session.execute(query) 
    print(f'Table {table_name} successfully created')
except Exception as e:
    print(e)


#### Insert Data

In [ ]:
query = f"INSERT INTO {table_name} \
        (song, userId, user) \
        VALUES (%s, %s, %s)"

for row in data.itertuples():
        try:
            session.execute(query, (row.song,
                                    int(row.userId),
                                    row.firstName + " " + row.lastName))
        except Exception as e:
            print(e)    

#### Select our results

In [ ]:
query = f"select user from {table_name} where song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
rows._current_rows

### Drop the tables before closing out the sessions

In [ ]:
tables = ['songs_by_session', 'songs_by_session_and_user', 'songs_by_title']
for table in tables:
    try:
        session.execute(f'drop table {table}')
        print(f'Dropped table {table}')
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()

# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Apache Cassandra coding portion of your project. 

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS data_modeling_project 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('data_modeling_project')
except Exception as e:
    print(e)

#### Read CSV file into a dataframe

In [ ]:
data = pd.read_csv('event_datafile_new.csv')
data.columns

#### Create pandas factory for converting Cassandra query results to df

In [ ]:
# Note I was trying to figure out a simple way to convert Cassandra query results to a dataframe and 
# found this on StackOverflow so I am attributing this
#https://stackoverflow.com/questions/41247345/python-read-cassandra-data-into-pandas
# This is the only thing I looked up during this project. It's pretty slick. 
    
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory

# Query 1
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4



#### Create Table

In [ ]:
# For this query I'll create a table with a composite key of sessionId and itemInSession
# Note I am not choosing to store all the data in this table but you could easily add the 
# other attributes

table_name = 'songs_by_session'

try:
    session.execute(f'drop table {table_name}')
except Exception as e:
    pass

query = f"create table if not exists {table_name} \
    (sessionId int, itemInSession int, artist varchar, song varchar, length float, \
    PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query) 
    print(f'Table {table_name} successfully created')
except Exception as e:
    print(e)

#### Insert Data

In [ ]:
query = f"INSERT INTO {table_name} \
        (sessionId, itemInSession, artist, song, length) \
        VALUES (%s, %s, %s, %s, %s)"

for row in data.itertuples():
        try:
            session.execute(query, (int(row.sessionId), 
                                    int(row.itemInSession), 
                                    row.artist, 
                                    row.song, 
                                    float(row.length)))
        except Exception as e:
            print(e)    

#### Select our results

In [ ]:
session_id = 338
item_id = 4

query = f'select artist, song, length from {table_name} where \
sessionId = {session_id} and itemInSession = {item_id}'

try:
    rows = session.execute(query)
except Exception as e:    
    print(e)

rows._current_rows

# Query 2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Create Table

In [ ]:
# For this query we'll create a table with a composite key of sessionId, userId, and itemInSession
# Note I am not choosing to store all the data in this table but you could easily add the 
# other attributes

table_name = 'songs_by_session_and_user'

try:
    session.execute(f'drop table {table_name}')
except Exception as e:
    pass

query = f"create table if not exists {table_name} \
    (sessionId int, userId int, itemInSession int,artist varchar, song varchar, user varchar, \
    PRIMARY KEY (sessionId, userId, itemInSession))"

try:
    session.execute(query) 
    print(f'Table {table_name} successfully created')
except Exception as e:
    print(e)
                        

#### Insert Data

In [ ]:
query = f"INSERT INTO {table_name} \
        (sessionId, userId, itemInSession, artist, song, user) \
        VALUES (%s, %s, %s, %s, %s, %s)"

for row in data.itertuples():
        try:
            session.execute(query, (int(row.sessionId),
                                    int(row.userId),
                                    int(row.itemInSession), 
                                    row.artist, 
                                    row.song, 
                                    row.firstName + " " + row.lastName))
        except Exception as e:
            print(e)    
            

#### Select our results

In [ ]:
user_id=10
session_id=182

query = f"select artist, song, user from {table_name} where \
    userId={user_id} and sessionId={session_id}"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
rows._current_rows

### Query 3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Create Table

In [ ]:
# For this query we'll create a table with a composite key of songtitle and userId . Note we
# need userID to make primary key unique since multiple users can listen to the same song.
# also note the same user could listen to the same song in multiple sessions, but since we don't 
# care about that we can choose to just have a primary key on these two fields.  Otherwise
# we could get duplicates in our query.  
# Also note I am choosing to store minimum data that we need for this query.  In reality I 
# would probabably store more attributes - better to have them then have to add them later.

table_name = 'songs_by_title'

try:
    session.execute(f'drop table {table_name}')
except Exception as e:
    pass

query = f"create table if not exists {table_name} \
    (song varchar, userId int, user varchar, \
    PRIMARY KEY (song, userId))"

try:
    session.execute(query) 
    print(f'Table {table_name} successfully created')
except Exception as e:
    print(e)


#### Insert Data

In [ ]:
query = f"INSERT INTO {table_name} \
        (song, userId, user) \
        VALUES (%s, %s, %s)"

for row in data.itertuples():
        try:
            session.execute(query, (row.song,
                                    int(row.userId),
                                    row.firstName + " " + row.lastName))
        except Exception as e:
            print(e)    

#### Select our results

In [ ]:
query = f"select user from {table_name} where song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
rows._current_rows

### Drop the tables before closing out the sessions

In [ ]:
tables = ['songs_by_session', 'songs_by_session_and_user', 'songs_by_title']
for table in tables:
    try:
        session.execute(f'drop table {table}')
        print(f'Dropped table {table}')
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()